In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

from utils import *

In [16]:
from sklearn.linear_model import LogisticRegression

classifier evidence over time for each subject / wedding
- train
- eval

In [35]:
np.array([1.,2.]).astype(int)

array([1, 2])

In [36]:
def read_logfile(sub_num):
  """ 
  read psychopy logifle into dataframe
  """
  # initialize dataframe
  df = pd.DataFrame(columns=['tstamp','logtype','logdata'])
  # open logfile
  fpath = 'behav/silvy_buckets/sub1%iday2/%i_viewing.log'%(sub_num,sub_num)
  f = open(fpath, "r")
  # loop over logfile rows
  first_TR_bool = True
  for x in f:
    tstamp,B,C = x.split('\t')
    tstamp = float(tstamp)
    # correct tstamp relative to first TR
    if first_TR_bool and C == 'Keypress: equal\n':
      first_TR_bool = False
      first_TR_tstamp = tstamp
    df.loc[tstamp,'logtype'] = B[:-1]
    df.loc[tstamp,'logdata'] = C[:-1]
    df.loc[tstamp,'tstamp'] = tstamp
  return df,first_TR_tstamp

def reformat_index(df,first_TR_tstamp,TR_rate=1.5):
  """
  include TR column
  reset timestamps relative to first TR
  reindex by row number
  """
  # add TR column
  df['TR'] = int(0)
  # reindex relative to first TR
  df.index = df.index - first_TR_tstamp 
  # include TR column
  df.loc[:,'TR'] = np.floor(df.index /TR_rate).astype(int)
  # remove negative TRs
  df = df[df.TR >= 0 ]
  # reindex by row number 
  df.index = np.arange(len(df))
  return df
  
def include_state_column(df):
  """ 
  include column "state" which indicates which stimulus is being shown
  """
  df['state'] = np.nan
  vid_strL = ['vid1a','vid1b','vid2','vid3','vid4','vid5']
  for vid_str0,vid_str1 in zip(vid_strL[:-1],vid_strL[1:]):
    vid_start_idx = df[df.logdata == '%s: autoDraw = True'%vid_str0].index
    vid_end_idx = df[df.logdata == '%s: autoDraw = True'%vid_str1].index
    for trial_num in range(10):
      df.loc[vid_start_idx[trial_num]:vid_end_idx[trial_num],'state']=vid_str0
  return df

In [37]:
def load_logdf(sub_num):
  """ wrapper for loading subject experiment timing info
  """ 
  ## read psychopy log file into dataframe
  df,first_TR_tstamp = read_logfile(sub_num)
  ##
  df = reformat_index(df,first_TR_tstamp)
  ##
  df = include_state_column(df)
  return df

In [41]:
df = load_logdf(33)
len(df[df.state == 'vid4'].TR)

189

In [19]:
df = load_logdf(33)
df.iloc[500:550]

,tstamp,logtype,logdata,TR,state
500,502.174,EXP,vid1a: autoDraw = False,316.0,vid1a
501,502.19,EXP,vid1b: autoDraw = True,316.0,vid1b
502,502.276,DATA,Keypress: equal,316.0,vid1b
503,503.776,DATA,Keypress: equal,317.0,vid1b
504,505.278,DATA,Keypress: equal,318.0,vid1b
505,506.778,DATA,Keypress: equal,319.0,vid1b
506,508.278,DATA,Keypress: equal,320.0,vid1b
507,509.778,DATA,Keypress: equal,321.0,vid1b
508,511.177,EXP,vid1b: autoDraw = False,322.0,vid1b
509,511.19,EXP,vid2: autoDraw = True,322.0,vid2


In [6]:
for sub_num in range(30,39):
  df = load_logdf(sub_num)
  print('******')
  print(df.iloc[500:550])

******
      tstamp  type                 logdata     TR  state
500  516.195  DATA         Keypress: equal  314.0  vid1a
501  517.226   EXP  vid1b: autoDraw = True  315.0  vid1b
502  517.702  DATA         Keypress: equal  315.0  vid1b
503    519.2  DATA         Keypress: equal  316.0  vid1b
504  520.702  DATA         Keypress: equal  317.0  vid1b
505  522.194  DATA         Keypress: equal  318.0  vid1b
506  523.694  DATA         Keypress: equal  319.0  vid1b
507  525.195  DATA         Keypress: equal  320.0  vid1b
508  526.009   EXP      Set vid1b finished  321.0  vid1b
509  526.226   EXP   vid2: autoDraw = True  321.0   vid2
510  526.694  DATA         Keypress: equal  321.0   vid2
511  528.194  DATA         Keypress: equal  322.0   vid2
512  529.694  DATA         Keypress: equal  323.0   vid2
513  531.194  DATA         Keypress: equal  324.0   vid2
514  532.694  DATA         Keypress: equal  325.0   vid2
515  534.194  DATA         Keypress: equal  326.0   vid2
516  535.694  DATA      

******
      tstamp  type                  logdata     TR  state
500  502.174   EXP  vid1a: autoDraw = False  316.0  vid1a
501   502.19   EXP   vid1b: autoDraw = True  316.0  vid1b
502  502.276  DATA          Keypress: equal  316.0  vid1b
503  503.776  DATA          Keypress: equal  317.0  vid1b
504  505.278  DATA          Keypress: equal  318.0  vid1b
505  506.778  DATA          Keypress: equal  319.0  vid1b
506  508.278  DATA          Keypress: equal  320.0  vid1b
507  509.778  DATA          Keypress: equal  321.0  vid1b
508  511.177   EXP  vid1b: autoDraw = False  322.0  vid1b
509   511.19   EXP    vid2: autoDraw = True  322.0   vid2
510  511.276  DATA          Keypress: equal  322.0   vid2
511  512.775  DATA          Keypress: equal  323.0   vid2
512  514.276  DATA          Keypress: equal  324.0   vid2
513  515.776  DATA          Keypress: equal  325.0   vid2
514  517.282  DATA          Keypress: equal  326.0   vid2
515  518.776  DATA          Keypress: equal  327.0   vid2
516  52

******
      tstamp  type                  logdata     TR  state
500  511.134  DATA          Keypress: equal  314.0  vid1a
501  512.644  DATA          Keypress: equal  315.0  vid1a
502   513.04   EXP  vid1a: autoDraw = False  316.0  vid1a
503  513.057   EXP   vid1b: autoDraw = True  316.0  vid1b
504  514.134  DATA          Keypress: equal  316.0  vid1b
505  515.636  DATA          Keypress: equal  317.0  vid1b
506  517.135  DATA          Keypress: equal  318.0  vid1b
507  518.634  DATA          Keypress: equal  319.0  vid1b
508  518.924   EXP       Set vid1b finished  320.0  vid1b
509  520.144  DATA          Keypress: equal  320.0  vid1b
510  521.644  DATA          Keypress: equal  321.0  vid1b
511  522.057   EXP    vid2: autoDraw = True  322.0   vid2
512  523.134  DATA          Keypress: equal  322.0   vid2
513  524.644  DATA          Keypress: equal  323.0   vid2
514  526.135  DATA          Keypress: equal  324.0   vid2
515  527.634  DATA          Keypress: equal  325.0   vid2
516  52

# load roi

In [ ]:
A = load_sub4d(30,task='videos',max_len=2000,numpy_output=False)

In [ ]:
"sub-130_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii"

In [7]:
def load_sub_roi(sub_n,roi_name):
  fpath = "sub-%i_%s.npy"%(sub_n,roi_name)
  return np.load('fmri_data/masked/'+fpath)

def load_group_roi(roi_name,max_len=1160):
  L = []
  for sub_n in SUB_NS:
    sub_roi = load_sub_roi(sub_n,roi_name)
    print(sub_roi.shape)
    L.append(sub_roi[:max_len,:])
  return np.array(L)

In [12]:
group_roi = load_group_roi(ROI_NAME_L[0])

(1168, 868)
(1166, 868)
(1168, 868)
(1167, 868)
(1166, 868)
(1168, 868)
(1166, 868)
(1167, 868)
(1166, 868)
